In [ ]:
# Facial Recognition System using TensorFlow and OpenCV
# ======================================================
#
# This notebook demonstrates a complete pipeline for facial detection and recognition.
# It uses:
#   - OpenCV Haar Cascade for face detection.
#   - A convolutional neural network (CNN) built with TensorFlow/Keras to recognize faces.
#
# The CNN is trained from scratch on the LFW (Labeled Faces in the Wild) dataset.
# After training, the system allows the user to upload an image,
# detect faces within it, and classify each face into one of the known identities.

# 1. Import necessary libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 2. Load the LFW dataset (only consider people with at least 50 images)
lfw_people = fetch_lfw_people(min_faces_per_person=50, resize=0.4)
X = lfw_people.images  # grayscale images
y = lfw_people.target  # numeric labels
target_names = lfw_people.target_names  # names corresponding to labels

print("Dataset shape:", X.shape)
print("Number of classes:", len(target_names))
print("Classes:", target_names)

# 3. Preprocess the data
# Expand dimension to add channel (needed for CNN input)
X = np.expand_dims(X, -1)
# Normalize pixel values to [0,1]
X = X.astype('float32') / 255.0

# 4. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert labels to categorical (one-hot encoding)
num_classes = len(target_names)
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

# 5. Build the CNN model for face recognition
input_shape = X_train.shape[1:]  # e.g., (50, 37, 1)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 6. Train the model
# For demonstration purposes, we use a small number of epochs.
epochs = 10
batch_size = 32

history = model.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, validation_split=0.1)

# Evaluate the model on test set
test_loss, test_acc = model.evaluate(X_test, y_test_cat)
print("Test accuracy: {:.2f}%".format(test_acc * 100))

# 7. Save the trained model (optional)
model.save('face_recognition_model.h5')

# 8. Download Haar Cascade file for face detection (if not already present)
!wget -q https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

# 9. Initialize the Haar Cascade classifier
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# 10. Function to detect faces and perform recognition on an input image
def detect_and_recognize(image_path):
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Unable to read image!")
        return
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    print("Detected {} face(s)".format(len(faces)))

    # For each detected face, perform recognition
    for (x, y, w, h) in faces:
        # Extract face ROI and resize to match model's expected input size
        face_roi = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face_roi, (input_shape[1], input_shape[0]))
        face_resized = face_resized.astype('float32') / 255.0
        face_resized = np.expand_dims(face_resized, axis=-1)
        face_resized = np.expand_dims(face_resized, axis=0)  # add batch dimension

        # Predict identity using the CNN
        pred = model.predict(face_resized)
        class_idx = np.argmax(pred, axis=1)[0]
        label = target_names[class_idx]
        confidence = pred[0][class_idx]

        # Draw rectangle and label on the original image
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        text = "{}: {:.2f}%".format(label, confidence * 100)
        cv2.putText(image, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (36,255,12), 2)

    # Convert image from BGR to RGB for display
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 8))
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.title("Face Detection & Recognition")
    plt.show()

# 11. Testing the system: upload an image and perform detection and recognition
from google.colab import files
uploaded = files.upload()

# Assume the user uploads one image, get its filename.
if uploaded:
    image_filename = list(uploaded.keys())[0]
    detect_and_recognize(image_filename)
